# SQLAlchemy: relacionamento 1–N (mínimo)

**Ideia rápida**
- O lado **N** guarda a **FK**: `ForeignKey("users.id")`.
- `relationship()` cria os atributos Python para navegar: `User.posts` ↔ `Post.author`.
- `back_populates` deve apontar para o **nome do atributo do outro lado**.
- Modelo típico: **User (1) — Post (N)**.



In [1]:
from sqlalchemy.orm import declarative_base, relationship
from sqlalchemy import Column, Integer, String, ForeignKey

Base = declarative_base()

class User(Base):
    __tablename__ = "users"
    id   = Column(Integer, primary_key=True)
    name = Column(String, nullable=False)
    posts = relationship("Post", back_populates="author")

class Post(Base):
    __tablename__ = "posts"
    id      = Column(Integer, primary_key=True)
    title   = Column(String, nullable=False)
    user_id = Column(Integer, ForeignKey("users.id"), nullable=False)
    author  = relationship("User", back_populates="posts")

In [7]:
from sqlalchemy import create_engine, select
from sqlalchemy.orm import sessionmaker

engine = create_engine("sqlite+pysqlite:///:memory:", echo=False, future=True)
Base.metadata.create_all(engine)

SessionLocal = sessionmaker(bind=engine, autoflush=False, autocommit=False)
session = SessionLocal()

In [8]:
# Criar
ana = User(name="Ana")
ana.posts.append(Post(title="Olá"))
session.add(ana)
session.commit()

In [9]:
# 1) Ligando pelo objeto (relationship)
bob = User(name="Bob")
p1 = Post(title="Primeiro do Bob", author=bob)  # seta o .author
session.add_all([bob, p1])
session.commit()

In [ ]:
# 2) Ligando pela FK direta (user_id)
carol = User(name="Carol")
session.add(carol); 
session.commit()  # precisa do id persistido

p2 = Post(title="Primeiro da Carol", user_id=carol.id)
session.add(p2)
session.commit()

In [6]:
session.close()